In [25]:
import os
import json
import shutil

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object


In [55]:
series_paths = get_series_path("/media/deeplearning/Elements/send_to_validated_dicom")
export_folder = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RELEVANCE_json'

In [57]:
len(series_paths)

262

In [60]:
not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []
secondary_series = []
localizer_series = []

for serie_path in series_paths:
    print(series_paths.index(serie_path))
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        remove_bi_file(serie_path)
        if( dicom_serie.is_image_modality() == False):
            not_image_folder.append(serie_path)
        elif (dicom_serie.is_series_valid() == False):
            non_valid_series.append(serie_path)
        elif(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
            non_attenuation_corrected.append(serie_path)
        elif(dicom_serie.is_localizer_series() == True):
            localizer_series.append(serie_path)
        else:

            write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
            print("json_file")
    except Exception as err:
        print(err)
        print(dicomsInfo)

_file
191
json_file
192
json_file
193
json_file
194
json_file
195
json_file
196
json_file
197
json_file
198
json_file
199
json_file
200
json_file
201
json_file
202
json_file
203
json_file
204
json_file
205
json_file
206
json_file
207
json_file
208
json_file
209
json_file
210
json_file
211
json_file
212
json_file
213
json_file
214
json_file
215
json_file
216
json_file
217
json_file
218
json_file
219
json_file
220
json_file
221
json_file
222
json_file
223
json_file
224
json_file
225
json_file
226
json_file
227
json_file
228
json_file
229
json_file
230
json_file
231
json_file
232
json_file
233
json_file
234
json_file
235
json_file
236
json_file
237
json_file
238
json_file
239
json_file
240
json_file
241
json_file
242
json_file
243
json_file
244
json_file
245
json_file
246
json_file
247
json_file
248
json_file
249
json_file
250
json_file
251
json_file
252
json_file
253
json_file
254
json_file
255
json_file
256
json_file
257
json_file
258
json_file
259
json_file
260
json_file
261
json_file


In [ ]:
#Index of series we have treated in Relevance envoi 2
index = []
for serie_path in series_paths : 
    study_uid = serie_path.split('/')[-1]
    index.append(study_uid)

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/', 'index_RELEVANCE', index)

In [61]:
#print(not_image_folder)
#print(non_valid_series)
#print(non_attenuation_corrected)
#print(empty_folders)
#print(secondary_series)
#print(localizer_series)

print(len(not_image_folder))
print(len(non_valid_series))
print(len(non_attenuation_corrected))
print(len(empty_folders))
print(len(secondary_series))
print(len(localizer_series))

0
0
0
0
0
0


In [38]:
for localizer_serie in localizer_series:
    shutil.rmtree(localizer_serie)

for not_image in not_image_folder:
    shutil.rmtree(not_image)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)

for non_valid_serie in non_valid_series : 
    shutil.rmtree(non_valid_serie)

for secondary_serie in secondary_series : 
    shutil.rmtree(secondary_serie)

In [39]:
for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

In [40]:
#relevance envoi 2 
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_RELEVANCE.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [43]:
liste_json = os.listdir('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RELEVANCE_json')
print(len(liste_json))

951


In [ ]:
for uid in dataset : 
    for json_f in liste_json : 
        if uid in json_f : 
            print(uid)
            print(json_f)
            os.remove('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RELEVANCE_json'+'/'+json_f)

In [51]:
#keep index : 

update = os.listdir('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RELEVANCE_json')
index = []
for uid in update : 
    study_uid = uid[:-5]
    index.append(study_uid)
    

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92', 'index_part_2', index)

In [49]:
write_json_file("/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92", "wrong Series Relevance_2", json.dumps(non_valid_series))